# 2.2 读取数据集

创建测试 CSV 文件，保存在 data 目录中。

In [2]:
import os

os.makedirs(os.path.join('data'), exist_ok=True)
data_file = os.path.join('data', 'house_tiny.csv')
print(data_file)
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

data/house_tiny.csv


In [3]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

`loc`: 通过行、列的 **名称/标签** 来索引

`iloc`: 通过行、列的 **索引位置（数字）** 来寻找数据

In [4]:
# 检索第 2 行的数据
data.loc[1]      # data.loc[1, :] 同理

NumRooms       2.0
Alley          NaN
Price       106000
Name: 1, dtype: object

In [5]:
# 读取第 Alley 列的全部值
data.loc[:, "Alley"]

0    Pave
1     NaN
2     NaN
3     NaN
Name: Alley, dtype: object

In [6]:
# 同时定位 第 2 行 NumRooms 的值
data.loc[2, "NumRooms"]

4.0

In [7]:
# 第二列的值 切片规则：左闭右开
data.iloc[:, 1]

0    Pave
1     NaN
2     NaN
3     NaN
Name: Alley, dtype: object

In [8]:
# 1 ~ 2 列
inputs = data.iloc[:, 0:2]
# 第 3 列
outputs = data.iloc[:, 2]
print(inputs)
print("------------------------------")
print(outputs)

   NumRooms Alley
0       NaN  Pave
1       2.0   NaN
2       4.0   NaN
3       NaN   NaN
------------------------------
0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64


In [9]:
# fillna NaN 填充
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


使用 pandas 将 Alley 转为 2 个列：`Alley_Pave` 及 `Alley_nan`

如果 Alley 为 Pave，本列（Alley_Pave）值为 1，否则为 0。

这样便于后续的张量格式转换以及数据的处理（将数据全部转换为数值的形式）。

In [10]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


## 转换为张量格式

In [11]:
from mxnet import np

X, y = np.array(inputs.to_numpy(dtype=float)), np.array(outputs.to_numpy(dtype=float))
X, y

(array([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]], dtype=float64),
 array([127500., 106000., 178100., 140000.], dtype=float64))

## 练习题

练习 1：除缺失值最多的列：

In [14]:
sum = data.isna().sum()
print(sum)
print("----------------")
data = data.drop(columns=sum.index[sum.argmax()])
print(data)

NumRooms    2
Price       0
dtype: int64
----------------
    Price
0  127500
1  106000
2  178100
3  140000


练习 2：将预处理后的数据集转换为张量格式

In [15]:
data_numpy = np.array(data.to_numpy(dtype=float))
data_numpy

array([[127500.],
       [106000.],
       [178100.],
       [140000.]], dtype=float64)